# Introduction

Converts CRX to RNX

In [ ]:
link="""https://geoftp.ibge.gov.br/informacoes_sobre_posicionamento_geodesico/rbmc/dados/2021/236/rjni2361.zip"""
rbmcfile=link.split("/")[-1]
year=21

# crx2rnx

In [ ]:
!wget https://raw.githubusercontent.com/geospace-code/georinex/main/src/georinex/rnxcmp/source/crx2rnx.c
!gcc crx2rnx.c -o crx2rnx

--2021-11-01 19:42:46--  https://raw.githubusercontent.com/geospace-code/georinex/main/src/georinex/rnxcmp/source/crx2rnx.c
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36738 (36K) [text/plain]
Saving to: ‘crx2rnx.c’

crx2rnx.c           100%[===================>]  35.88K  --.-KB/s    in 0.004s  

2021-11-01 19:42:46 (8.06 MB/s) - ‘crx2rnx.c’ saved [36738/36738]



# Running

## Downloading RINEX 3 file from RBMC

In [ ]:
!wget "$link"

--2021-11-01 19:42:47--  https://geoftp.ibge.gov.br/informacoes_sobre_posicionamento_geodesico/rbmc/dados/2021/236/rjni2361.zip
Resolving geoftp.ibge.gov.br (geoftp.ibge.gov.br)... 191.233.136.86
Connecting to geoftp.ibge.gov.br (geoftp.ibge.gov.br)|191.233.136.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3521002 (3.4M) [application/zip]
Saving to: ‘rjni2361.zip’

rjni2361.zip        100%[===================>]   3.36M  1.34MB/s    in 2.5s    

2021-11-01 19:42:51 (1.34 MB/s) - ‘rjni2361.zip’ saved [3521002/3521002]



## Unzipping and transforming from hatanaka to rnx

In [ ]:
!unzip $rbmcfile


Archive:  rjni2361.zip
  inflating: rjni2361.21d            
  inflating: rjni2361.21n            
  inflating: rjni2361.21g            
  inflating: rjni2361.21l            


In [ ]:
#Check the file sizes
!ls -la 

total 15368
drwxr-xr-x 1 root root    4096 Nov  1 19:42 .
drwxr-xr-x 1 root root    4096 Nov  1 19:38 ..
drwxr-xr-x 4 root root    4096 Oct 26 13:33 .config
-rwxr-xr-x 1 root root   31408 Nov  1 19:42 crx2rnx
-rw-r--r-- 1 root root   36738 Nov  1 19:42 crx2rnx.c
-rw-r--r-- 1 root root 8055691 Aug 25 05:22 rjni2361.21d
-rw-r--r-- 1 root root  369037 Aug 25 22:30 rjni2361.21g
-rw-r--r-- 1 root root 3419763 Aug 25 22:30 rjni2361.21l
-rw-r--r-- 1 root root  274957 Aug 25 22:30 rjni2361.21n
-rw-r--r-- 1 root root 3521002 Aug 25 23:01 rjni2361.zip
drwxr-xr-x 1 root root    4096 Oct 26 13:34 sample_data


In [ ]:
crxfile=rbmcfile.replace(".zip",'.{}d'.format(year))
!./crx2rnx $crxfile
print("Output file should be {}. Please check the Files tab.".format("rjni2361.21d"[:-1]+"o"))

Output file should be rjni2361.21o. Please check the Files tab.


# Downloading TEQC

In [ ]:
!wget https://www.unavco.org/software/data-processing/teqc/development/teqc_CentOSLx86_64s.zip -O teqc.zip
!unzip teqc.zip 

--2021-11-01 19:42:52--  https://www.unavco.org/software/data-processing/teqc/development/teqc_CentOSLx86_64s.zip
Resolving www.unavco.org (www.unavco.org)... 69.44.86.106
Connecting to www.unavco.org (www.unavco.org)|69.44.86.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1087404 (1.0M) [application/zip]
Saving to: ‘teqc.zip’

teqc.zip            100%[===================>]   1.04M  1.18MB/s    in 0.9s    

2021-11-01 19:42:54 (1.18 MB/s) - ‘teqc.zip’ saved [1087404/1087404]

Archive:  teqc.zip
  inflating: teqc                    


# Saving the GPS only rinex 2 file to newRinex.obs

In [ ]:
rnx2file=crxfile[:-1]+"o"
newFile=rnx2file.replace(".","gps.")
!./teqc -E -C -R -S -O.obs L1L2L5C1P2C5S1S2S5 $rnx2file > $newFile
print("RINEX 2 file saved to {}".format(newFile))

RINEX 2 file saved to rjni2361gps.21o


In [ ]:
#Let's fix the lack of wavelength factor
!sed '/^.*APPROX POSITION XYZ.*/a \     1     1                                                WAVELENGTH FACT L1\/2' -i $newFile 

In [ ]:
!head $newFile -n 100

     2.11           OBSERVATION DATA    M (MIXED)           RINEX VERSION / TYPE
teqc  2019Feb25                         20211101 19:42:54UTCPGM / RUN BY / DATE
RBMC                IBGE/CGED                               OBSERVER / AGENCY
5750R51585          TRIMBLE NETR9       5.50                REC # / TYPE / VERS
1551129599          TRM115000.00    NONE                    ANT # / TYPE
        0.0080        0.0000        0.0000                  ANTENNA: DELTA H/E/N
  4289663.4011 -4018945.7498 -2467135.8382                  APPROX POSITION XYZ
     1     1                                                WAVELENGTH FACT L1/2
     9    L1    L2    L5    C1    P2    C5    S1    S2    S5# / TYPES OF OBSERV
    15.0000                                                 INTERVAL
Linux 2.6.32-573.12.1.x86_64|x86_64|gcc -static|Linux 64|=+ COMMENT
teqc  2019Feb25                         20210825 05:21:57UTCCOMMENT
CODIGO: 96321    Estacao: Niteroi UFF                       COMMENT
RJNI         